### 📌 Cell 5: Import Libraries

**Mục đích:** Nhập các thư viện cần thiết cho dự án

**Input:** Không có  
**Output:** In ra phiên bản OpenCV và NumPy

**Code làm gì:**
```python
import cv2              # OpenCV - xử lý video/hình ảnh
import numpy as np      # NumPy - tính toán mảng
import matplotlib.pyplot as plt  # Vẽ biểu đồ
from IPython.display import Video, Image, display  # Hiển thị media
import os               # Thao tác file/thư mục
```

**Ý nghĩa:**
- `cv2`: Thư viện chính để đọc video, xử lý ảnh, detect features
- `numpy`: Xử lý ma trận tọa độ điểm, tính toán vector
- `matplotlib`: Vẽ kết quả tracking, biểu đồ phân tích
- `IPython.display`: Hiển thị video output trong notebook
- `os`: Kiểm tra file tồn tại, lấy kích thước file

**Vai trò:** Chuẩn bị công cụ cần thiết cho toàn bộ dự án


### 📌 Cell 7: Configuration Parameters (Cấu hình tham số)

**Mục đích:** Thiết lập các tham số cho thuật toán KLT

**Input:** Không có  
**Output:** In "Configuration loaded successfully!"

**Code làm gì:**

**1. feature_params (Tham số phát hiện góc):**
```python
maxCorners=100       # Phát hiện tối đa 100 góc
qualityLevel=0.3     # Chỉ lấy góc có chất lượng ≥ 30%
minDistance=7        # Các góc cách nhau ít nhất 7 pixels
blockSize=7          # Kích thước vùng tính toán góc
```

**2. lk_params (Tham số Lucas-Kanade):**
```python
winSize=(15, 15)     # Cửa sổ tìm kiếm 15x15 pixels
maxLevel=2           # Dùng 3 tầng kim tự tháp (0,1,2)
criteria=(...)       # Dừng sau 10 lần lặp hoặc độ chính xác 0.03
```

**3. VIDEO_INPUT/OUTPUT:**
```python
VIDEO_INPUT = "Easy.mp4"                    # File video đầu vào
VIDEO_OUTPUT = "klt_tracking_output.avi"    # File video đầu ra
DISPLAY_REALTIME = False                    # Có hiển thị real-time không
FEATURE_REFRESH_THRESHOLD = 15              # Tái tạo features khi < 15 điểm
```

**Ý nghĩa:**
- `maxCorners`: Càng nhiều → track nhiều điểm hơn nhưng chậm hơn
- `qualityLevel`: Càng cao → chọn góc mạnh hơn, ít false positive
- `minDistance`: Tránh các điểm quá gần nhau, phân bố đều hơn
- `winSize`: Lớn hơn → track displacement lớn nhưng chậm hơn
- `maxLevel`: Tầng cao hơn → track chuyển động nhanh tốt hơn

**Vai trò:** Điều chỉnh độ nhạy và hiệu suất của thuật toán tracking

### 📌 Cell 9: Main KLT Tracking Function (Hàm chính)

**Mục đích:** Hàm thực hiện toàn bộ quá trình tracking KLT

**Input:**
- `video_path`: Đường dẫn video (VD: "Easy.mp4")
- `output_path`: Nơi lưu video kết quả
- `display`: True/False - hiển thị real-time
- `feature_refresh_threshold`: Ngưỡng tái tạo features

**Output:**
- `frames_with_tracking`: Danh sách frames đã vẽ tracking
- `statistics`: Thống kê (số frames, số lần refresh, fps...)

**Code làm gì (từng bước):**

**BƯỚC 1: Mở video và lấy thông tin**
```python
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)      # Lấy FPS
width = cap.get(...)                  # Lấy chiều rộng
height = cap.get(...)                 # Lấy chiều cao
total_frames = cap.get(...)           # Tổng số frames
```
→ Biết video có bao nhiêu frames, kích thước bao nhiêu

**BƯỚC 2: Tạo VideoWriter để lưu kết quả**
```python
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
```
→ Chuẩn bị để ghi video output

**BƯỚC 3: Đọc frame đầu và detect features**
```python
old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
old_points = cv2.goodFeaturesToTrack(old_gray, **feature_params)
```
→ Chuyển sang ảnh xám và tìm 100 góc mạnh nhất

**BƯỚC 4: Vòng lặp tracking (CORE ALGORITHM)**
```python
while True:
    # 4.1: Đọc frame mới
    new_frame = cap.read()
    new_gray = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
    
    # 4.2: Tính optical flow (TÌM ĐIỂM MỚI)
    new_points, status, error = cv2.calcOpticalFlowPyrLK(
        old_gray, new_gray, old_points, None, **lk_params
    )
    # → new_points: Vị trí mới của các điểm
    # → status: 1=tìm được, 0=mất dấu
    
    # 4.3: Lọc điểm tốt
    good_new = new_points[status == 1]  # Chỉ lấy điểm track được
    good_old = old_points[status == 1]
    
    # 4.4: VẼ MOTION VECTORS (Quiver plot)
    for new, old in zip(good_new, good_old):
        # Vẽ đường thẳng từ vị trí cũ → mới
        mask = cv2.line(mask, (old_x, old_y), (new_x, new_y), color, 2)
        # Vẽ chấm tròn tại vị trí hiện tại
        frame = cv2.circle(frame, (new_x, new_y), 5, color, -1)
    
    # 4.5: Ghép frame + motion trails
    output_frame = cv2.add(new_frame, mask)
    
    # 4.6: CẬP NHẬT cho frame tiếp theo
    old_gray = new_gray
    old_points = good_new
    
    # 4.7: TÁI TẠO FEATURES nếu mất quá nhiều
    if len(good_new) < 15:
        old_points = cv2.goodFeaturesToTrack(new_gray, ...)
        mask = np.zeros_like(old_frame)  # Xóa trails
```

**BƯỚC 5: Dọn dẹp và trả về kết quả**
```python
cap.release()
out.release()
return frames_with_tracking, statistics
```

**Ý nghĩa:**
- **calcOpticalFlowPyrLK**: Trái tim của KLT, tìm điểm di chuyển đến đâu
- **Pyramidal**: Xử lý nhiều scale → track chuyển động nhanh tốt hơn
- **Feature refresh**: Khi mất quá nhiều điểm → detect lại để tiếp tục

**Vai trò:** Thực hiện toàn bộ quy trình KLT tracking từ đầu đến cuối

### 📌 Cell 11: Run KLT Tracking (Chạy tracking)

**Mục đích:** Gọi hàm KLT để xử lý video

**Input:**
- `VIDEO_INPUT` = "Easy.mp4"
- `VIDEO_OUTPUT` = "klt_tracking_output.avi"
- `DISPLAY_REALTIME` = False
- `FEATURE_REFRESH_THRESHOLD` = 15

**Output:**
- Video được lưu vào `klt_tracking_output.avi`
- `frames`: List chứa tất cả frames đã tracking
- `stats`: Dict thống kê (total_frames, feature_refreshes, fps...)
- In ra console: "Tracking completed successfully!"

**Code làm gì:**
```python
frames, stats = klt_feature_tracking(
    video_path=VIDEO_INPUT,
    output_path=VIDEO_OUTPUT,
    ...
)
```

**Quá trình thực tế:**
1. Đọc video Easy.mp4 (có thể 200-400 frames)
2. Detect 100 features ở frame đầu
3. Track qua từng frame, vẽ motion vectors
4. Khi features < 15 → detect lại
5. Lưu tất cả vào file .avi
6. Mất ~30 giây - 2 phút tùy video

**Ý nghĩa:** Đây là cell quan trọng nhất - thực thi toàn bộ thuật toán

**Vai trò:** Biến lý thuyết thành kết quả thực tế (video output)

### 📌 Cell 13: Display Sample Frames (Hiển thị frames mẫu)

**Mục đích:** Xem một số frames đại diện từ video tracking

**Input:**
- `frames`: List frames từ cell 11 (VD: 200 frames)

**Output:**
- Figure 2x3 với 5 frames: đầu, 1/4, giữa, 3/4, cuối
- File ảnh: `klt_tracking_samples.png`

**Code làm gì:**
```python
# Chọn 5 frames đại diện
sample_indices = [0, len(frames)//4, len(frames)//2, 3*len(frames)//4, len(frames)-1]
# VD: [0, 50, 100, 150, 199]

# Vẽ từng frame
for idx, frame_idx in enumerate(sample_indices):
    frame_rgb = cv2.cvtColor(frames[frame_idx], cv2.COLOR_BGR2RGB)
    axes[idx].imshow(frame_rgb)
    axes[idx].set_title(f'Frame {frame_idx+1}')

plt.savefig('klt_tracking_samples.png')
```

**Ý nghĩa:**
- Thấy được tracking hoạt động như thế nào qua thời gian
- Kiểm tra xem features có bị mất không
- So sánh frame đầu vs cuối → độ drift

**Ví dụ output:**
```
┌─────────┬─────────┬─────────┐
│ Frame 1 │ Frame 51│ Frame101│  ← Đầu → Giữa
├─────────┼─────────┼─────────┤
│Frame151 │Frame199 │  (rỗng) │  ← 3/4 → Cuối
└─────────┴─────────┴─────────┘
```

**Vai trò:** Visualization để đánh giá chất lượng tracking

### 📌 Cell 18: Compare Shi-Tomasi vs Harris (So sánh 2 detector)

**Mục đích:** So sánh 2 phương pháp phát hiện góc

**Input:**
- Frame đầu tiên của video
- `feature_params` (tham số Shi-Tomasi)

**Output:**
- 3 ảnh: Original | Shi-Tomasi | Harris
- File: `feature_comparison.png`
- In số features detect được

**Code làm gì:**
```python
# Shi-Tomasi (mặc định)
corners_shi = cv2.goodFeaturesToTrack(gray, **feature_params)

# Harris (thêm tham số useHarrisDetector)
harris_params = feature_params.copy()
harris_params['useHarrisDetector'] = True
harris_params['k'] = 0.04
corners_harris = cv2.goodFeaturesToTrack(gray, **harris_params)

# Vẽ chấm tròn lên các góc detect được
for corner in corners_shi:
    cv2.circle(frame, (x, y), 5, (0,255,0), -1)  # Xanh lá
for corner in corners_harris:
    cv2.circle(frame, (x, y), 5, (255,0,0), -1)  # Đỏ
```

**Ý nghĩa:**

**Shi-Tomasi (Good Features to Track):**
- Tối ưu cho tracking
- Chọn góc "dễ track" nhất
- Response function: R = min(λ₁, λ₂)

**Harris Corner Detector:**
- Detector cổ điển hơn
- Response function: R = λ₁λ₂ - k(λ₁ + λ₂)²
- Tham số k ≈ 0.04-0.06

**Kết quả thường thấy:**
- Shi-Tomasi: Detect nhiều hơn, phân bố đều hơn
- Harris: Chọn lọc hơn, tập trung vào góc rất mạnh

**Vai trò:** Hiểu sự khác biệt giữa các corner detector